In [3]:
#importing the libraries
import json
import nltk
import numpy as np
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()#stemming

#creating the custom functions

# tokenize each word in the sentence
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

# stem and lower the word
def stem(word):
    return stemmer.stem(word.lower())


"""
splitting each word in the sentences and adding it to an array
If words exists in the sentence, it returns 1 else 0
example:
sentence = ["how", "are", "you", "today"]
words = ["hi", "hello", "how", "bye", "today", "welcome"]
It provides:
bag   = [  0 ,    0   ,   1  ,   0  ,    1   ,     0    ]
"""
def bag_of_words(tokenized_sentence, words):
    sentence_words = [stem(word) for word in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for index, w in enumerate(words):
        if w in sentence_words: 
            bag[index] = 1

    return bag

#loading the restaurants.json file
with open('restaurants.json', 'r') as f:
    restaurants = json.load(f)

allWords = []
tags = []
xy_data = []

# looping through every sentence in restaurant patterns
for restaurant in restaurants['restaurants']:
    tag = restaurant['tag']
    tags.append(tag)
    for pattern in restaurant['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)# adding to tag list
        allWords.extend(w)
        xy_data.append((w, tag))

#cleaning the data
ignore_words = ['.', '!', '?']
allWords = [stem(w) for w in allWords if w not in ignore_words]

# remove duplicate data and sorting
allWords = sorted(set(allWords))
tags = sorted(set(tags))

print("There are", len(xy_data), "patterns.")
print("There are", len(tags), "tags which are:", tags)
print("There are", len(allWords), "uniquly stemmed sorted words which are:", allWords)

There are 19 patterns.
There are 6 tags which are: ['cleanliness', 'drink', 'greet', 'offer', 'often', 'popular']
There are 36 uniquly stemmed sorted words which are: ['alcohol', 'ani', 'anyon', 'be', 'can', 'cleanli', 'do', 'drink', 'for', 'have', 'hello', 'hey', 'hi', 'how', 'i', 'is', 'it', 'maintain', 'mainten', 'meal', 'of', 'offer', 'often', 'oftenli', 'popular', 'restaur', 'sanit', 'serv', 'someth', 'the', 'there', 'to', 'want', 'what', 'you', 'your']


In [5]:
# create the training data
train_x = []
train_y = []
for (pattern_sentence, tag) in xy_data:
    # x: bag of words for each pattern_info
    bag = bag_of_words(pattern_sentence, allWords)
    train_x.append(bag)
    
    # y: PyTorch CrossEntropyLoss only needs class labels
    label = tags.index(tag)
    train_y.append(label)

train_x = np.array(train_x)
train_y = np.array(train_y)

class NeuralNetwork(nn.Module):
    def __init__(self, inputSize, hiddenSize, noClasses):
        super(NeuralNetwork, self).__init__()
        self.n1 = nn.Linear(inputSize, hiddenSize) 
        self.n2 = nn.Linear(hiddenSize, hiddenSize) 
        self.n3 = nn.Linear(hiddenSize, noClasses)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.n1(x)
        out = self.relu(out)
        out = self.n2(out)
        out = self.relu(out)
        out = self.n3(out)
        return out
    
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(train_x)
        self.x_data = train_x
        self.y_data = train_y

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples
    
# Hyper-parameters for Neural network
numEpoches = 1000
batch_size = 8
learningRate = 0.001
inputSize = len(train_x[0])
hiddenSize = 8
outputSize = len(tags)
print("Input size  :", inputSize)
print("Output size :", outputSize)

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNetwork(inputSize, hiddenSize, outputSize).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)

# Training the model
for epoch in range(numEpoches):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{numEpoches}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"modelState": model.state_dict(),
"inputSize": inputSize,
"hiddenSize": hiddenSize,
"outputSize": outputSize,
"allWords": allWords,
"tags": tags
}

#saving the trained model
filename = "filedata.pth"
torch.save(data, filename)

print(f'Training complete. File has been saved to {filename}')

Input size  : 36
Output size : 6
Epoch [100/1000], Loss: 1.2163
Epoch [200/1000], Loss: 0.6545
Epoch [300/1000], Loss: 0.0791
Epoch [400/1000], Loss: 0.0110
Epoch [500/1000], Loss: 0.0068
Epoch [600/1000], Loss: 0.0016
Epoch [700/1000], Loss: 0.0025
Epoch [800/1000], Loss: 0.0023
Epoch [900/1000], Loss: 0.0017
Epoch [1000/1000], Loss: 0.0009
final loss: 0.0009
Training complete. File has been saved to filedata.pth


In [6]:
#loading the saved model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('restaurants.json', 'r') as json_data:
    restaurants = json.load(json_data)

filename = "filedata.pth"
data = torch.load(filename)

inputSize = data["inputSize"]
hiddenSize = data["hiddenSize"]
outputSize = data["outputSize"]
allWords = data['allWords']
tags = data['tags']
modelState = data["modelState"]

model = NeuralNetwork(inputSize, hiddenSize, outputSize).to(device)
model.load_state_dict(modelState)
model.eval()


NeuralNetwork(
  (n1): Linear(in_features=36, out_features=8, bias=True)
  (n2): Linear(in_features=8, out_features=8, bias=True)
  (n3): Linear(in_features=8, out_features=6, bias=True)
  (relu): ReLU()
)

In [7]:
#use of the chatbot
botname = "Buddy"
print("Welcome to the restaurant. I am your restaurant buddy.")
print("You can enter exit to finish the conversation anytime.")
while True:
    sentence = input("You  : ")
    if sentence == "exit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, allWords)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for restaurant in restaurants['restaurants']:
            if tag == restaurant["tag"]:
                print(f"{botname}: {random.choice(restaurant['replys'])}")
    else:
        print(f"{botname}: I do not understand what you want to say.")

Welcome to the restaurant. I am your restaurant buddy.
You can enter exit to finish the conversation anytime.
You  : Hi
Buddy: What can I do for you?
You  : Hey
Buddy: How can I help
You  : I want something to drink
Buddy: You can have wine or beer
You  : What is restaurant popular for?
Buddy: It is popular for Chicken Biryani
You  : Any offer to me as dish
Buddy: We provide you wine and beer
You  : exit
